# Gráficos CEPAL

In [1]:
import xml.etree.ElementTree as ET

tree_tamano_medio_hogares = ET.parse('Tamano_medio_hogares.xml')
root = tree_tamano_medio_hogares.getroot()

In [2]:
dims = ['País', 'Área geográfica', 'Quintil', 'Años']

identifiers  = {k : {} for k in dims}

for elem in root[2].iter():
    d = elem.attrib
    try:
        if d['name'] in dims:
            key = d['name']
            continue
        else:
            identifiers[key][d['id']] = d['name']
    except KeyError:
        pass
    

In [3]:
dict_tamano_medio_hogares = {}

dim_id = {'dim_208' : 'País',
         'dim_326' : 'Área geográfica',
         'dim_378' : 'Quintil',
         'dim_29117' : 'Años',
         'id_fuente' : 'id_fuente',
         'ids_notas' : 'ids_notas',
         'iso3' : 'iso3',
         'valor' : 'valor'}

for i, elem in enumerate(root[1].iter()):
    if i > 0:
        if i == 1:
            for key, item in elem.attrib.items():
                k = dim_id[key]
                try: 
                    dict_tamano_medio_hogares[k] = [identifiers[k][item]]
                except KeyError:
                    dict_tamano_medio_hogares[k] = [item]
        else:
            for key, item in elem.attrib.items():
                k = dim_id[key]
                try:
                    dict_tamano_medio_hogares[k].append(identifiers[k][item])
                except KeyError:
                    dict_tamano_medio_hogares[k].append(item)

In [38]:
import pandas as pd

data_frame = pd.DataFrame(dict_tamano_medio_hogares)

data_frame.head(20)

,País,Área geográfica,Quintil,Años,id_fuente,ids_notas,iso3,valor
0,América Latina (promedio simple),Nacional,Quintil 1,2001,26,"8620,7765",,5.2
1,América Latina (promedio simple),Nacional,Quintil 2,2001,26,"8620,7765",,4.8
2,América Latina (promedio simple),Nacional,Quintil 3,2001,26,"8620,7765",,4.3
3,América Latina (promedio simple),Nacional,Quintil 4,2001,26,"8620,7765",,3.8
4,América Latina (promedio simple),Nacional,Quintil 5,2001,26,"8620,7765",,3.2
5,América Latina (promedio simple),Nacional,Total quintiles,2001,26,"8620,7765",,4.3
6,América Latina (promedio simple),Rural,Quintil 1,2001,26,"8620,10395",,5.7
7,América Latina (promedio simple),Rural,Quintil 2,2001,26,"8620,10395",,5.2
8,América Latina (promedio simple),Rural,Quintil 3,2001,26,"8620,10395",,4.8
9,América Latina (promedio simple),Rural,Quintil 4,2001,26,"8620,10395",,4.2


In [80]:
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

data_frame["Años"] = data_frame["Años"].astype(dtype='intc')
data_frame["valor"] = data_frame["valor"].astype(dtype='float64')

#User input
desagregacion = ["Área geográfica", "Quintil"]
paises = list(data_frame['País'].unique())

#Desagregaciones
anios = sorted(list(map(int,data_frame['Años'].unique())))
years = [2002,2010,2019]
quintiles = list(data_frame["Quintil"].unique())
area = list(data_frame["Área geográfica"].unique())

filt = data_frame[
    (data_frame["País"]=="América Latina (promedio simple)") &
    (data_frame["Área geográfica"]=="Nacional") &
    (data_frame["Años"].isin(years))
    ]

fig = go.Figure(data=[
go.Bar(name=str(year),
       x=quintiles,
       y=filt[data_frame["Años"]==year]['valor']) for year in years
])

fig.update_layout(title_text='América Latina (promedio simple)',
                  barmode='group')
fig.show()

In [5]:
data_frame[['Años','valor']] = data_frame[['Años','valor']].astype(dtype='float64')

filtered_year = data_frame[(data_frame['Años']==2002) |
                           (data_frame['Años']==2010) |
                           (data_frame['Años']==2019) ] 

filtered_year.to_csv('Databases/tamano_hogar.csv', index=False)
#gr_pais_v = data_frame.groupby(['País','Quintil']).mean()

#gr_pais_v.head(114)

In [81]:
#pip install dash dash-renderer dash-html-components dash-core-components plotly

import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

app = dash.Dash()

app.layout= html.Div(children=[
    html.H1('Tamaño medio del hogar'), #Header 1 - Children 1
    dcc.Dropdown( #Dropdown
        id='input_country', #Identify dash component in callback
        options = [{'label' : c, 'value' : c} for c in paises], #dropdown options
        placeholder ='Seleccionar País o Región'
    ),  #Output - Children 3
    dcc.Dropdown( #Dropdown
        id='input_dim', 
        options = [{'label' : c, 'value' : c} for c in ["Quintil", "Área geográfica"]], 
        placeholder ='Seleccionar desagregación'
    ),
    dcc.Graph(id='Graph')
    ]) 

@app.callback(
    Output('Graph','figure'),
    Input('input_country','value'),
    Input('input_dim', 'value')
    )

def update_graph(country, dim):

    if dim == 'Quintil':
        
        filt = data_frame[
            (data_frame["País"]==country) &
            (data_frame["Área geográfica"]=="Nacional") &
            (data_frame["Años"].isin(years))
            ]
        
        x_d = quintiles
        
    else:
        
        filt = data_frame[
            (data_frame["País"]==country) &
            (data_frame["Quintil"]=="Total quintiles") &
            (data_frame["Años"].isin(years))
            ]
            
        x_d = area
            
    fig = go.Figure(data=[
    go.Bar(name=str(year),
           x=x_d,
           y=filt[data_frame["Años"]==year]['valor']) for year in years
    ])

    fig.update_layout(title_text=country,
                      barmode='group')
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [64]:
dcc.Dropdown?